<a href="https://colab.research.google.com/github/xpeng-26/Erdos_Finance_LLM_Project/blob/Jerry_project/Notebooks/train_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
!git clone https://github.com/xpeng-26/Erdos_Finance_LLM_Project.git

Cloning into 'Erdos_Finance_LLM_Project'...
remote: Enumerating objects: 272, done.
remote: Counting objects: 100% (272/272), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 272 (delta 124), reused 237 (delta 89), pack-reused 0 (from 0)
Receiving objects: 100% (272/272), 6.13 MiB | 10.61 MiB/s, done.
Resolving deltas: 100% (124/124), done.


In [22]:
%cd Erdos_Finance_LLM_Project/src

/content/Erdos_Finance_LLM_Project/src/Erdos_Finance_LLM_Project/src


In [7]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

/bin/bash: line 1: nvidia-smi: command not found


In [18]:
import os
import sys
from datetime import datetime

# Add ../src to the path
proj_path = os.path.abspath(os.path.join(os.getcwd(), "../src"))
sys.path.insert(0, proj_path)

# Import the parse config function to parse the .toml file
from utils.config_tool import parse_config
from utils.logging_tool import initialize_logger
from strategy.train_trading_agent import train_trading_agent

# Get the time
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Load the config file
config = parse_config("../config/trade_w_llm.toml")
# Initialize the logger
log_file = "{}_log_{}.txt".format(
			'training_agent',
			str(timestamp))
logger = initialize_logger(log_path=os.path.join(config['info']['local_data_path'], 'logs'),log_file=log_file)

# Train the trading agent
train_trading_agent(config, logger)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/xiaokangwang/Documents/PycharmProjects/Projects for Erdos 2025 spring/dataset/logs/training_agent_log_20250327_225150.txt'